# setup environment

In [4]:
from scripts.dm4_to_png import save_dm4_BF_to_png
from labelling.binary_search_labeler import label
from src.run import main
from src.inference import plot_embeddings


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fintan-hardy (fintan-hardy-imperial-college-london). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /rds/general/user/fgh24/home/.netrc


## STEPS 
- Turn dm4 into png images 
- label data 
- Train model on labelled data
- Do inference

In [ ]:
# IMPORTANT
dm4_path = "../data/dm4/boracite_Diffraction_SI.dm4"
output_png_path = "data/png/2025-05-30-test"
labelling_path = "labels/2025-05-30-test_label.yaml"
config_file = "configs/2025-05-30-test.yaml"
output_save_path = "output/"

dim1 = 512  # y-axis
dim2 = 512  # x-axis

# labelling hyperperameters
sampling_space = 10
number_of_labels_to_assign = 100

In [9]:
# read config file and print output
with open(config_file, "r") as file:
    config = file.read()
    print(config)

seed: 42
test_only: False
accelerator: "cuda"

data:
  data_dir: "data/png/2025-05-30-test"
  train_dir: "data/png/2025-05-30-test"
  data_loader: "RawPngLoader"
  train_ratio: 0.9
  val_ratio: 0.05
  test_ratio: 0.05
  num_workers: 4
  augment: False
  image_size: 256
  labels_file: "labels.yaml"

wandb:
  project_name: "polarisation_mapping-live"
  run_name: "first run experiment"
  experiment_id: "1"
  ex_description: "Running first experiment to test the pipeline with simple model"

model:
  in_channels: 1
  batch_size: 8
  model_name: "ThreeLayerCNN"
  accuracy_metric: "cross_entropy"

optimizer:
  optimizer: "adamw"
  lr: 0.0004
  weight_decay: 0.002
  momentum: 0.9
  betas: [0.9, 0.999]

scheduler:
  scheduler: "plateau"
  step_size: 30
  step_size_up: 1000
  patience: 2
  factor: 0.5
  min_lr: 3e-5
  max_lr: 5e-5
  T_max: 10
  gamma: 0.1

trainer:
  num_nodes: 1
  devices: 1
  max_epochs: 100
  log_dir: "./logs"
  log_every_n_steps: 100
  load_path: ""
  resume_from: False 
  c

# Turn dm4 into .png

In [10]:
save_dm4_BF_to_png(dm4_path, labelling_path, crop=False, crop_values=None)

File is not a valid DM3 or DM4


# Label Images

In [ ]:
label(output_png_path, labelling_path, sampling_space, number_of_labels_to_assign)

# Training

In [ ]:
main(config_file)

# Inference

In [ ]:
plot_embeddings(
    config_file,
    dim1,
    dim2,
    save_path=output_save_path,
)